# Flower Quickstart (Simulation with TensorFlow/Keras)

Welcome to Flower, a friendly federated learning framework!

In this notebook, we'll simulate a federated learning system with 100 clients. The clients will use TensorFlow/Keras to define model training and evaluation. Let's start by installing Flower Nightly, published as `flwr-nightly` on PyPI:

In [1]:
# !pip install git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]  # For a specific branch (release/0.17) w/ extra ("simulation")
# # !pip install -U flwr["simulation"]  # Once 0.17.1 is released

Next, we import the required dependencies. The most important imports are Flower (`flwr`) and TensorFlow:

In [3]:
import os
import math

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl


from simulation import start_simulation
import torch
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split




ModuleNotFoundError: No module named 'src'

With that out of the way, let's move on to the interesting bits. Federated learning systems consist of a server and multiple clients. In Flower, we create clients by implementing subclasses of `flwr.client.Client` or `flwr.client.NumPyClient`. We use `NumPyClient` in this tutorial because it is easier to implement and requires us to write less boilerplate.

To implement the Flower client, we create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`:

- `get_parameters`: Return the current local model parameters
- `fit`: Receive model parameters from the server, train the model parameters on the local data, and return the (updated) model parameters to the server 
- `evaluate`: Received model parameters from the server, evaluate the model parameters on the local data, and return the evaluation result to the server

We mentioned that our clients will use TensorFlow/Keras for the model training and evaluation. Keras models provide methods that make the implementation staightforward: we can update the local model with server-provides parameters through `model.set_weights`, we can train/evaluate the model through `fit/evaluate`, and we can get the updated model parameters through `model.get_weights`.

Let's see a simple implementation:

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, data, y_train, x_val, y_val) -> None:
        # self.model = model
        # self.x_train, self.y_train = x_train, y_train
        # self.x_val, self.y_val = x_train, y_train
        self.data = data

    def get_parameters(self):
        return np.zeros(1)

    def fit(self, parameters, config):
        # self.model.set_weights(parameters)
        # self.model.fit(self.x_train, self.y_train, epochs=1, verbose=2)
        return np.zeros(1), 0, {}

    def evaluate(self, parameters, config):
        learning_rate = torch.exp(config['h0']).item()
        learning_rate_decay = torch.exp(config['h1']).item()
        l2_regular = torch.exp(config['h2']).item()
        s = int(config['seed'])

        config = tf.ConfigProto()
        tf.disable_v2_behavior()
        tf.reset_default_graph()
        tf.set_random_seed(s)
        np.random.seed(s)
        
        num_classes = 10
        epochs = 20
        X_train, X_test, Y_train, Y_test = self.data[s][0], self.data[s][1], self.data[s][2], self.data[s][3]

        X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
        X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train = 1 - X_train
        X_test = 1 - X_test

        dropout_rate = 0.0

        batch_size = 32
        conv_filters = 16
        dense_units = 8

        num_conv_layers = 2
        kernel_size = 3
        pool_size = 3

        # build the CNN model using Keras
        model = Sequential()
        model.add(Conv2D(conv_filters, (kernel_size, kernel_size), padding='same',
                         input_shape=X_train.shape[1:], kernel_regularizer=regularizers.l2(l2_regular)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
        model.add(Dropout(dropout_rate))
        model.add(Flatten())
        model.add(Dense(dense_units, kernel_regularizer=regularizers.l2(l2_regular)))
        model.add(Activation('relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))

        opt = RMSprop(lr=learning_rate, decay=learning_rate_decay)
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
        #
        history = model.fit(X_train, Y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(X_test, Y_test),
                      shuffle=True, verbose=0)
        val_acc = max(history.history['val_acc'])  
      
        return 0.0, s, {"accuracy": val_acc}


Our class `FlowerClient` defines how local training/evaluation will be performed and allows Flower to call the local training/evaluation through `fit` and `evaluate`. Each instance of `FlowerClient` represents a *single client* in our federated learning system. Federated learning systems have multiple clients (otherwise there's not much to federate, is there?), so each client will be represented by its own instance of `FlowerClient`. If we have, for example, three clients in our workload, we'd have three instances of `FlowerClient`. Flower calls `FlowerClient.fit` on the respective instance when the server selects a particular client for training (and `FlowerClient.evaluate` for evaluation).

In this notebook, we want to simulate a federated learning system with 100 clients on a single machine. This means that the server and all 100 clients will live on a single machine and share resources such as CPU, GPU, and memory. Having 100 clients would mean having 100 instances of `FlowerClient` im memory. Doing this on a single machine can quickly exhaust the available memory resources, even if only a subset of these clients participates in a single round of federated learning.

In addition to the regular capabilities where server and clients run on multiple machines, Flower therefore provides special simulation capabilities that create `FlowerClient` instances only when they are actually necessary for training or evaluation. To enable the Flower framework to create clients when necessary, we need to implement a function called `client_fn` that creates a `FlowerClient` instance on demand. Flower calls `client_fn` whenever it needs an instance of one particular client to call `fit` or `evaluate` (those instances are usually discarded after use). Clients are identified by a client ID, or short `cid`. The `cid` can be used, for example, to load different local data partitions for each client:

In [ ]:
def get_client_fn():
    data = pickle.load(open("/home/ubuntu/r244_alex/R244_Project/src/examples_code/quickstart_simulation/data/emnist_data_mixed.pkl", "rb"))
    def client_fn(cid: str) -> fl.client.Client:
        nonlocal data
        # Create model
        # model = tf.keras.models.Sequential(
        #     [
        #         tf.keras.layers.Flatten(input_shape=(28, 28)),
        #         tf.keras.layers.Dense(128, activation="relu"),
        #         tf.keras.layers.Dropout(0.2),
        #         tf.keras.layers.Dense(10, activation="softmax"),
        #     ]
        # )
        # model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

        # # Load data partition (divide MNIST into NUM_CLIENTS distinct partitions)
        # (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
        # partition_size = math.floor(len(x_train) / NUM_CLIENTS)
        # idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
        # x_train_cid = x_train[idx_from:idx_to] / 255.0
        # y_train_cid = y_train[idx_from:idx_to]

        # # Use 10% of the client's training data for validation
        # split_idx = math.floor(len(x_train) * 0.9)
        # x_train_cid, y_train_cid = x_train_cid[:split_idx], y_train_cid[:split_idx]
        # x_val_cid, y_val_cid = x_train_cid[split_idx:], y_train_cid[split_idx:]

        # Create and return client
        return FlowerClient(0, data, 0, 0, 0)
    return client_fn

In [ ]:
client_fn = get_client_fn()
NUM_CLIENTS=10


client_fn("123").evaluate(None,{
    "h0":torch.Tensor([np.log(0.001)]),
    "h1":torch.Tensor([np.log(0.001)]),
    "h2":torch.Tensor([np.log(0.001)]), 
    "seed":1,
})

(0.0, 1, {'accuracy': 0.82758623})

We now have `FlowerClient` which defines client-side training and evaluation and `client_fn` which allows Flower to create `FlowerClient` instances whenever it needs to call `fit` or `evaluate` on one particular client. The last step is to start the actual simulation using `flwr.simulation.start_simulation`. 

The function `start_simulation` accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy. The strategy encapsulates the federated learning approach/algorithm, for example, *Federated Averaging* (FedAvg).

Flower comes with a number of built-in strategies, but we can also use our own strategy implementations to customize nearly all aspects of the federated learning approach. For this example, we use the built-in `FedAvg` implementation and customize it using a few basic parameters. The last step is the actual call to `start_simulation` which - you guessed it - actually starts the simulation.

In [ ]:
client_fn = get_client_fn()
# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 10% of available clients for training
        fraction_eval=1.0,  # Sample 5% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Never sample less than 10 clients for training
        min_eval_clients=NUM_CLIENTS,  # Never sample less than 5 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait until at least 75 clients are available
)

# Start simulation
start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=1,
    strategy=strategy,
)

Congratulations! With that, you built a Flower client, customized it's instantiation through the `client_fn`, customized the server-side execution through a `FedAvg` strategy configured for this workload, and started a simulation with 100 clients (each holding their own individual partition of the MNIST dataset).

Next, you can continue to explore more advanced Flower topics:

- Deploy server and clients on different machines using `start_server` and `start_client`
- Customize the server-side execution through custom strategies
- Customize the client-side exectution through `config` dictionaries